In [8]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import NoSuchElementException
import re
import time
import pymongo
from pymongo import MongoClient

In [9]:
client = MongoClient('localhost', 27017)
db = client.db_world_bank
reports_coll = db["reports"]


In [4]:
start_urls_list = {
    'Journals' : 'https://openknowledge.worldbank.org/handle/10986/2210/discover?filtertype=supportedlanguage&filter_relational_operator=equals&filter=en',
    'Serial Publications': 'https://openknowledge.worldbank.org/handle/10986/11863/discover?filtertype=supportedlanguage&filter_relational_operator=equals&filter=en',
    'Technical Papers':'https://openknowledge.worldbank.org/handle/10986/18022/discover?filtertype=supportedlanguage&filter_relational_operator=equals&filter=en',
    'Economic and Sector Work':'https://openknowledge.worldbank.org/handle/10986/6/discover?filtertype=supportedlanguage&filter_relational_operator=equals&filter=en',
    'Working papers':'https://openknowledge.worldbank.org/handle/10986/8/discover?rpp=10&etal=0&group_by=none&page=1&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en',
    'Knowledge Notes':'https://openknowledge.worldbank.org/handle/10986/9387/discover?filtertype=supportedlanguage&filter_relational_operator=equals&filter=en'
}

In [5]:
def add_to_dict(arg):
    try:
        arg
    except NoSuchElementException:
        pass


def get_all_elements_on_page(driver):
    try:
        title = driver.find_element_by_xpath("//*[@class='ds-div-head']").text
    except NoSuchElementException:
        title = ''
        pass
    try:
        abstract = driver.find_element_by_xpath("//*[@class='okr-item-page-field-wrapper abstract']").text
    except NoSuchElementException:
        abstract = ''
        pass 
    try:   
        abstract_views = driver.find_element_by_xpath("//*[@class='item-stat abstract-views']").text.split("\n")[1]
    except NoSuchElementException:
        abstract_views = ''
        pass
    try:
        file_downloads = driver.find_element_by_xpath("//*[@class='item-stat file-downloads']").text.split("\n")[1]
    except NoSuchElementException:
        file_downloads = ''
        pass
    try:
        published_date = driver.find_element_by_xpath("//*[@class='simple-item-view-other word-break']").text
    except NoSuchElementException:
        published_date = ''
        pass
    try:
        pdf_link = driver.find_element_by_xpath("//*[@class='bitstream-link']").get_attribute('href')
    except NoSuchElementException:
        pdf_link = ''
        pass
    
    dict_document = {'title':title,'abstract':abstract,'abstract_views':abstract_views,
                     'file_downloads':file_downloads, 'published_date' : published_date, 'pdf_link':pdf_link,
                     'collection':''}
    return dict_document
    


In [ ]:
options = Options()
options.headless = True
driver = webdriver.Firefox()

collection = 'Serial Publications'
temp_url = 'https://openknowledge.worldbank.org/handle/10986/11863/discover?rpp=10&etal=0&group_by=none&page=79&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en'
driver.get(temp_url)
#driver.get(start_urls_list[collection])
time.sleep(3)

## Find number pages in collection
last_page_link = driver.find_element_by_xpath("//*[@class='last-page-link']")
child_element_split_url = last_page_link.find_elements_by_xpath(".//*")[0].get_attribute('href').split('&')
for element in child_element_split_url:
    if 'page' in element:
        total_pages_numb = int(element.split("=")[1])


print(total_pages_numb)
n=0
for results_page in range(total_pages_numb-1):
    n+=1
    print(n)
    button_next_page = driver.find_element_by_xpath("//*[@class='next-page-link']").get_attribute('href')
    print(button_next_page)
    # get list links on page
    items_on_page = driver.find_elements_by_xpath("//*[@class='item-wrapper']")
    root_url = 'https://openknowledge.worldbank.org/'
    lst_links_documents = [root_url + element.get_attribute('data-href') for element in items_on_page]

    for document_link in lst_links_documents:
        driver.get(document_link)
        time.sleep(3)
        dict_infos = get_all_elements_on_page(driver)
        dict_infos['collection'] = collection
        reports_coll.insert_one(dict_infos)
    # move to next page
    driver.get(button_next_page)
    time.sleep(3)



182
1
https://openknowledge.worldbank.org/handle/10986/11863/discover?rpp=10&etal=0&group_by=none&page=80&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
2
https://openknowledge.worldbank.org/handle/10986/11863/discover?rpp=10&etal=0&group_by=none&page=81&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
3
https://openknowledge.worldbank.org/handle/10986/11863/discover?rpp=10&etal=0&group_by=none&page=82&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
4
https://openknowledge.worldbank.org/handle/10986/11863/discover?rpp=10&etal=0&group_by=none&page=83&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
5
https://openknowledge.worldbank.org/handle/10986/11863/discover?rpp=10&etal=0&group_by=none&page=84&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
6
https://openknowledge.worldbank.org/handle/10986/11863/discover?rpp=10&etal=0&group_by=no

46
https://openknowledge.worldbank.org/handle/10986/11863/discover?rpp=10&etal=0&group_by=none&page=125&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
47
https://openknowledge.worldbank.org/handle/10986/11863/discover?rpp=10&etal=0&group_by=none&page=126&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
48
https://openknowledge.worldbank.org/handle/10986/11863/discover?rpp=10&etal=0&group_by=none&page=127&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
49
https://openknowledge.worldbank.org/handle/10986/11863/discover?rpp=10&etal=0&group_by=none&page=128&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
50
https://openknowledge.worldbank.org/handle/10986/11863/discover?rpp=10&etal=0&group_by=none&page=129&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
51
https://openknowledge.worldbank.org/handle/10986/11863/discover?rpp=10&etal=0&grou